<a href="https://colab.research.google.com/github/mohamedbenhouad-cell/chapitre_3/blob/main/Corrig%C3%A9_TP_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Encodage des variables qualitatives

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# Charger les données depuis le fichier Excel
file_path = "/content/TP_3.2.xlsx"  # Remplacez par votre chemin réel
df = pd.read_excel(file_path)
# Supprimer les espaces cachés dans les noms de colonnes
df.columns = df.columns.str.strip()
# Vérifier les colonnes disponibles
print("\nColonnes disponibles dans le fichier :", df.columns.tolist())
#  Encodage des variables qualitatives
#  Encodage nominal de X2 (emploi) et X3 (matrimonial) avec LabelEncoder
if 'X2' in df.columns:
    label_encoder_X2 = LabelEncoder()
    df['X2'] = label_encoder_X2.fit_transform(df['X2'].astype(str).str.strip())
if 'X3' in df.columns:
    label_encoder_X3 = LabelEncoder()
    df['X3'] = label_encoder_X3.fit_transform(df['X3'].astype(str).str.strip())
# Encodage ordinal pour X4 (éducation)
education_order = {'primary': 0, 'secondary': 1, 'tertiary': 2}
if 'X4' in df.columns:
    df['X4'] = df['X4'].astype(str).str.strip().map(education_order)
# 3️ Encodage binaire pour X5, X7, X8 et Y (yes → 1, no → 0)
binary_columns = ['X5', 'X7', 'X8', 'Y']
binary_mapping = {'yes': 1, 'no': 0}  # Correction pour les variables binaires et Y
for col in binary_columns:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().map(binary_mapping)
#  Vérification de l'encodage
print("\n Encodage effectué avec succès !")
print("\nDonnées après encodage :")
print(df.head())
# Sauvegarde du fichier encodé
output_file = "/content/TP_3.2_encoded.xlsx"
df.to_excel(output_file, index=False)
print(f"\n Fichier encodé sauvegardé sous : {output_file}")


# **2.1.Régression logistique**

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score
# Chargement des données Train et Test
file_path = "/content/TP_3.2_encoded.xlsx"  # <-- votre fichier Excel
df_train = pd.read_excel(file_path, sheet_name="Train")
df_test  = pd.read_excel(file_path, sheet_name="Test")  # <-- AJOUT TEST ICI
# Variables
features = ['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10']
target = "Y"
# Séparation X / y (avec .copy() pour éviter les SettingWithCopyWarning)
X_train = df_train[features].copy()
y_train = df_train[target].copy()
X_test  = df_test[features].copy()
y_test  = df_test[target].copy()
# Normalisation
scaler = StandardScaler()
quantitative_columns = ['X1', 'X6', 'X9', 'X10']
X_train.loc[:, quantitative_columns] = scaler.fit_transform(X_train[quantitative_columns])
X_test.loc[:,  quantitative_columns] = scaler.transform(X_test[quantitative_columns])
# 3) Régression logistique
logistic_model = LogisticRegression(max_iter=1000, solver="lbfgs")  # multi-classe OK
logistic_model.fit(X_train, y_train)
# Coefficients + p-values (statsmodels)
X_train_const = sm.add_constant(X_train, has_constant="add")
logit_model = sm.Logit(y_train, X_train_const)
result = logit_model.fit(disp=False)
coef_pvalues_df = pd.DataFrame({
    "Variable": ["Intercept"] + X_train.columns.tolist(),
    "Coefficient": result.params.values,
    "P-value": result.pvalues.values
})
print("\nTableau des coefficients et des p-values :")
print(coef_pvalues_df)
# Prédiction sur Test + Évaluation
y_pred = logistic_model.predict(X_test)
df_test["Y_pred"] = y_pred
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted", zero_division=0)
recall = recall_score(y_test, y_pred, average="weighted", zero_division=0)
f1 = f1_score(y_test, y_pred, average="weighted", zero_division=0)
f2 = fbeta_score(y_test, y_pred, beta=2, average="weighted", zero_division=0)
print("\nÉvaluation sur les données de test avec la régression logistique :")
print("Accuracy :", accuracy)
print("Précision :", precision)
print("Recall :", recall)
print("F1-score :", f1)
print("F2-score :", f2)

# **2.2.Machines à vecteurs de support (SVM) **

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score
# Chargement des données d'apprentissage à partir de la feuille Excel
df_train = pd.read_excel('/content/TP_3.2_encoded.xlsx', sheet_name='Train')
# Sélection des variables exogènes (X1 à X10) et de la variable cible (Y)
X_train = df_train[['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10']]
y_train = df_train['Y']
# Normalisation des variables quantitatives (X1, X6, X9, X10)
scaler = StandardScaler()
quantitative_columns = ['X1', 'X6', 'X9', 'X10']
X_train[quantitative_columns] = scaler.fit_transform(X_train[quantitative_columns])
# Application du modèle SVM linéaire
svm_linear_model = SVC(kernel='linear', C=1.0)  # SVM avec noyau linéaire
svm_linear_model.fit(X_train, y_train)
# Chargement des données de test à partir de la feuille Excel
df_test = pd.read_excel('/content/TP_3.2_encoded.xlsx', sheet_name='Test')

# Sélection des variables exogènes (X1 à X10) et de la variable cible (Y)
X_test = df_test[['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10']]
y_test = df_test['Y']
# Normalisation des variables quantitatives pour les données de test
X_test[quantitative_columns] = scaler.transform(X_test[quantitative_columns])
# Prédictions sur la base de données de test
y_pred = svm_linear_model.predict(X_test)
# Ajout de la colonne Y_pred avec les prédictions dans df_test
df_test['Y_pred'] = y_pred
# Évaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
f2 = fbeta_score(y_test, y_pred, beta=2, average='weighted')
# Affichage des métriques d'évaluation
print("\nÉvaluation sur les données de test avec SVM linéaire :")
print("Accuracy :", accuracy)
print("Précision :", precision)
print("Recall :", recall)
print("F1-score :", f1)
print("F2-score :", f2)


# **2.3.Naïve Bayes Gaussien**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score
# Chargement des données d'apprentissage à partir de la feuille Excel
df_train = pd.read_excel('/content/TP_3.2_encoded.xlsx', sheet_name='Train')
# Sélection des variables exogènes (X1 à X10) et de la variable cible (Y)
X_train = df_train[['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10']]
y_train = df_train['Y']
# Application du modèle Naïve Bayes Gaussien
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
# Chargement des données de test à partir de la feuille Excel
df_test = pd.read_excel('/content/TP_3.2_encoded.xlsx', sheet_name='Test')
# Sélection des variables exogènes (X1 à X10) et de la variable cible (Y)
X_test = df_test[['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10']]
y_test = df_test['Y']
# Prédictions sur la base de données de test
y_pred = nb_model.predict(X_test)
# Ajout de la colonne Y_pred avec les prédictions dans df_test
df_test['Y_pred'] = y_pred
# Évaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
f2 = fbeta_score(y_test, y_pred, beta=2, average='weighted')
# Affichage des métriques d'évaluation
print("\nÉvaluation sur les données de test avec Naïve Bayes Gaussien :")
print("Accuracy :", accuracy)
print("Précision :", precision)
print("Recall :", recall)
print("F1-score :", f1)
print("F2-score :", f2)


# **2.4.Arbres de décision avec le coefficient GINI et une profondeur maximale de 3**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score
import graphviz
from sklearn.tree import export_graphviz
# Chargement des données d'apprentissage à partir de la feuille Excel
df_train = pd.read_excel('/content/TP_3.2_encoded.xlsx', sheet_name='Train')
# Sélection des variables exogènes (X1 à X10) et de la variable cible (Y)
X_train = df_train[['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10']]
y_train = df_train['Y']
# Application du modèle Arbre de Décision
tree_model = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=42)
tree_model.fit(X_train, y_train)
# Chargement des données de test à partir de la feuille Excel
df_test = pd.read_excel('/content/TP_3.2_encoded.xlsx', sheet_name='Test')
# Sélection des variables exogènes (X1 à X10) et de la variable cible (Y)
X_test = df_test[['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10']]
y_test = df_test['Y']
# Prédictions sur la base de données de test
y_pred = tree_model.predict(X_test)
# Ajout de la colonne Y_pred avec les prédictions dans df_test
df_test['Y_pred'] = y_pred
# Évaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
f2 = fbeta_score(y_test, y_pred, beta=2, average='weighted')
# Affichage des métriques d'évaluation
print("\nÉvaluation sur les données de test avec Arbre de Décision :")
print("Accuracy :", accuracy)
print("Précision :", precision)
print("Recall :", recall)
print("F1-score :", f1)
print("F2-score :", f2)
# Affichage de l'arbre de décision en mode texte
plt.figure(figsize=(15, 8))
plot_tree(tree_model, feature_names=X_train.columns, class_names=True, filled=True)
plt.show()
# Exporter l'arbre en format Graphviz
dot_data = export_graphviz(tree_model, out_file=None,
                           feature_names=X_train.columns,
                           class_names=[str(cls) for cls in tree_model.classes_],
                           filled=True, rounded=True, special_characters=True)
# Affichage avec Graphviz
graph = graphviz.Source(dot_data)
graph.render("arbre_decision")  # Sauvegarde sous forme d'image
graph.view()


# **2.5.Forêts aléatoires avec l’affichage de l’importance des caractéristiques**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score
# Chargement des données d'apprentissage à partir de la feuille Excel
df_train = pd.read_excel('/content/TP_3.2_encoded.xlsx', sheet_name='Train')
# Sélection des variables exogènes (X1 à X10) et de la variable cible (Y)
X_train = df_train[['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10']]
y_train = df_train['Y']
# Application du modèle Forêt Aléatoire
rf_model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
rf_model.fit(X_train, y_train)
# Chargement des données de test à partir de la feuille Excel
df_test = pd.read_excel('/content/TP_3.2_encoded.xlsx', sheet_name='Test')
# Sélection des variables exogènes (X1 à X10) et de la variable cible (Y)
X_test = df_test[['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10']]
y_test = df_test['Y']
# Prédictions sur la base de données de test
y_pred = rf_model.predict(X_test)
# Ajout de la colonne Y_pred avec les prédictions dans df_test
df_test['Y_pred'] = y_pred
# Évaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
f2 = fbeta_score(y_test, y_pred, beta=2, average='weighted')
# Affichage des métriques d'évaluation
print("\nÉvaluation sur les données de test avec Forêt Aléatoire :")
print("Accuracy :", accuracy)
print("Précision :", precision)
print("Recall :", recall)
print("F1-score :", f1)
print("F2-score :", f2)
# Affichage de l'importance des variables
feature_importances = rf_model.feature_importances_
feature_names = X_train.columns
plt.figure(figsize=(10, 6))
plt.barh(feature_names, feature_importances, color='skyblue')
plt.xlabel("Importance")
plt.ylabel("Variables")
plt.title("Importance des variables dans le modèle Random Forest")
plt.show()
